In [8]:
import respy as rp
import numpy as np

In [9]:
params, options = rp.get_example_model("kw_94_one", with_data=False)
simulate = rp.get_simulate_func(params, options)

We are now ready to study the effect of ambiguity on average educational attainment.

In [10]:
for eta in np.linspace(0, 0.01, 11):
    params.loc[("eta", "eta"), :] = eta
    df = simulate(params)
    stat = df.groupby("Identifier")["Experience_Edu"].max().mean()
    print(eta, stat)

0.0 12.603
0.001 13.427
0.002 13.775
0.003 14.025
0.004 14.285
0.005 14.478
0.006 14.686
0.007 14.838
0.008 15.005
0.009000000000000001 15.144
0.01 15.32


We need to determine whether we have a sound implementation of robust decision making setup up and thus investigate the a modified version of the KW model that includes some level of ambiguity.

* We estimate a model by maximum likelihood using all true values as the starting point.

* We estimate a model with only $\eta$ perturbed.

* We estimate a model with risk-only, $\eta = 0$ fixed, to see how different the likelihood is at the estimated parameter values.